In [10]:
import pandas as pd
import numpy
import os
import datetime
import plotly.express as px

print(os.chdir('./'))


df = pd.read_csv('../data/Walmart_Store_sales.csv',low_memory=False)
print(df.head())
print(df.shape)
print(df.dtypes)

None
   Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0    6.0  18-02-2011    1572117.54           NaN        59.61       3.045   
1   13.0  25-03-2011    1807545.43           0.0        42.38       3.435   
2   17.0  27-07-2012           NaN           0.0          NaN         NaN   
3   11.0         NaN    1244390.03           0.0        84.57         NaN   
4    6.0  28-05-2010    1644470.66           0.0        78.89       2.759   

          CPI  Unemployment  
0  214.777523         6.858  
1  128.616064         7.470  
2  130.719581         5.936  
3  214.556497         7.346  
4  212.412888         7.092  
(150, 8)
Store           float64
Date             object
Weekly_Sales    float64
Holiday_Flag    float64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object


150 lignes pour 8 colonnes, petit volume de données 

In [11]:
print("Description des données : ")
print(df.columns)
print(df.describe())

Description des données : 
Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment'],
      dtype='object')
            Store  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
count  150.000000  1.360000e+02    138.000000   132.000000  136.000000   
mean     9.866667  1.249536e+06      0.079710    61.398106    3.320853   
std      6.231191  6.474630e+05      0.271831    18.378901    0.478149   
min      1.000000  2.689290e+05      0.000000    18.790000    2.514000   
25%      4.000000  6.050757e+05      0.000000    45.587500    2.852250   
50%      9.000000  1.261424e+06      0.000000    62.985000    3.451000   
75%     15.750000  1.806386e+06      0.000000    76.345000    3.706250   
max     20.000000  2.771397e+06      1.000000    91.650000    4.193000   

              CPI  Unemployment  
count  138.000000    135.000000  
mean   179.898509      7.598430  
std     40.274956      1.577173  
min    126.111903      5.143000 

# Exploration des données

In [12]:
print("On explore les données...")
print(df.isna().sum())

On explore les données...
Store            0
Date            18
Weekly_Sales    14
Holiday_Flag    12
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
dtype: int64


# Data Cleaning

In [4]:
print("Affichage des du nombre de valeurs manquantes par colonnes :")
print(df.isna().sum())

Affichage des du nombre de valeurs manquantes par colonnes :
Store            0
Date            18
Weekly_Sales    14
Holiday_Flag    12
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
dtype: int64


In [5]:
print("On drop les Weekly_Sales à zéro qui est la valeur à déduire pour éviter les biais")
new_df = df.copy()
print(f"Shape de données est : {new_df.shape}")

new_df = new_df.dropna(subset="Weekly_Sales")
print(f"La nouvelle shape de données est : {new_df.shape}")

new_df = new_df.fillna(new_df.mean(numeric_only=True))
print(new_df.isna().sum())

print(f"La nouvelle shape de données est : {new_df.shape}")
print(new_df.isna().sum())
print(f"La nouvelle shape de données est : {new_df.shape}")



On drop les Weekly_Sales à zéro qui est la valeur à déduire pour éviter les biais
Shape de données est : (150, 8)
La nouvelle shape de données est : (136, 8)
Store            0
Date            18
Weekly_Sales     0
Holiday_Flag     0
Temperature      0
Fuel_Price       0
CPI              0
Unemployment     0
dtype: int64
La nouvelle shape de données est : (136, 8)
Store            0
Date            18
Weekly_Sales     0
Holiday_Flag     0
Temperature      0
Fuel_Price       0
CPI              0
Unemployment     0
dtype: int64
La nouvelle shape de données est : (136, 8)


## Il manque toujours 18 dates
Je décide de les supprimer 

In [6]:
new_df = new_df.dropna(how='any')
print(f"La nouvelle shape de données est : {new_df.shape}")

La nouvelle shape de données est : (118, 8)


In [7]:
print("Conversion des dates")
#Changement simple des infos car bien formaté à la base
new_df['Date'] = pd.to_datetime(new_df['Date'])
print(new_df['Date'].dtype)
print(new_df['Date'])
new_df['year'] = new_df['Date'].dt.year
new_df['month'] = new_df['Date'].dt.month
new_df['day'] = new_df['Date'].dt.day
new_df['day_of_week'] = new_df['Date'].dt.dayofweek
new_df.head()

Conversion des dates
datetime64[ns]
0     2011-02-18
1     2011-03-25
4     2010-05-28
5     2010-05-28
6     2011-06-03
         ...    
144   2012-10-19
145   2010-06-18
147   2010-06-11
148   2011-08-12
149   2012-04-20
Name: Date, Length: 118, dtype: datetime64[ns]


C:\Users\bonna\AppData\Local\Temp\ipykernel_30844\52456130.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  new_df['Date'] = pd.to_datetime(new_df['Date'])


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,day_of_week
0,6.0,2011-02-18,1572117.54,0.072,59.610000,3.045,214.777523,6.858,2011,2,18,4
1,13.0,2011-03-25,1807545.43,0.000,42.380000,3.435,128.616064,7.470,2011,3,25,4
4,6.0,2010-05-28,1644470.66,0.000,78.890000,2.759,212.412888,7.092,2010,5,28,4
5,4.0,2010-05-28,1857533.70,0.000,60.853967,2.756,126.160226,7.896,2010,5,28,4
6,15.0,2011-06-03,695396.19,0.000,69.800000,4.069,134.855161,7.658,2011,6,3,4


In [8]:
print("drop des outliers")
print(f"Shape du dataFrame : {new_df.shape}")
outliers_col = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
for col in outliers_col:
    mean = new_df[col].mean()
    ecart_type = new_df[col].std()
    bas, haut = mean - 3*ecart_type, mean + 3*ecart_type
    new_df = new_df[(new_df[col]>= bas) & (new_df[col] <= haut)]
print(f"Shape du dataFrame après supp des outliers : {new_df.shape}")



drop des outliers
Shape du dataFrame : (118, 12)
Shape du dataFrame après supp des outliers : (113, 12)


In [9]:
print("enregistrement du nouveau DataFrame ...")
print(os.listdir())
new_df.to_csv('../data/clean_data.csv')

enregistrement du nouveau DataFrame ...
['eda.ipynb']


In [ ]:
print("Distribution des données : C.A par année")
fig_bar = px.bar(new_df.groupby("year")["Weekly_Sales"].sum(),title="C.A par année")
fig_bar.show()

Distribution des données : C.A par année


In [76]:

new_df_month = new_df.groupby(["month","year"])["Weekly_Sales"].sum().reset_index()
# new_df_month["month"] = new_df_month["month"].astype(str)
plot_month = px.line(new_df_month,x="month",
    y="Weekly_Sales",
    color="year",

    title="Revenus mensuels")
plot_month.show()
plot_month = px.line(new_df.groupby(["month","year"])["Fuel_Price"].sum().reset_index(),x='month',y="Fuel_Price",color="year")
plot_month.show()

In [83]:
print("correlation")
correlation = new_df[['Weekly_Sales',"Temperature",'Fuel_Price', 'CPI', 'Unemployment',]].corr('pearson')
print(correlation)
corr_df_fig = px.imshow(correlation, text_auto=True,
    color_continuous_scale="RdBu_r",
    title="Matrice de corrélation (Pearson)",
    zmin=-1, zmax=1)
corr_df_fig.show()

correlation
              Weekly_Sales  Temperature  Fuel_Price       CPI  Unemployment
Weekly_Sales      1.000000    -0.188149   -0.017168 -0.337314      0.163012
Temperature      -0.188149     1.000000   -0.054591  0.150712     -0.185821
Fuel_Price       -0.017168    -0.054591    1.000000 -0.153558     -0.020948
CPI              -0.337314     0.150712   -0.153558  1.000000     -0.152158
Unemployment      0.163012    -0.185821   -0.020948 -0.152158      1.000000
